In [3]:
import requests 
import pymysql
from bs4 import BeautifulSoup

In [7]:

def db_config(host,port,user,password,db,table):
    conn = pymysql.connect(host='localhost',port=3306,user='root',password='')
    cur  = conn.cursor()
    query = " CREATE DATABASE IF NOT EXISTS gold_price;"
    cur.execute(query)
    if table=='history' :
        query = """    create table IF NOT EXISTS  `{}`.`{}`(
                        id int primary key auto_increment,
                        date text NOT NULL,
                        price text
                    ) DEFAULT CHARSET=utf8 DEFAULT COLLATE utf8_general_ci;
                """.format(db,table)
    else :
        query = """    create table IF NOT EXISTS  `{}`.`{}`(
                        id int primary key auto_increment,
                        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        Unit text NOT NULL,
                        price text
                    ) DEFAULT CHARSET=utf8 DEFAULT COLLATE utf8_general_ci;
                """.format(db,table)
        
    cur.execute(query)
    conn.close()
    return "conected and schema and tables created or founded"

In [8]:
def insert_history(db,table,date,price):
    query = """INSERT INTO `{}`.`{}`(date,price) 
    VALUES(%s,%s)""".format(db,table)
    val   = (date,price)
    conn  =  pymysql.connect(host='localhost',port=3306,user='root',password='')
    cur   = conn.cursor()
    cur.execute(query,val)
    conn.commit()
    conn.close()
    return "Done"

In [9]:
def insert_today(db,table,date,price):
    query = """INSERT INTO `{}`.`{}`(Unit,price) 
    VALUES(%s,%s)""".format(db,table)
    val   = (date,price)
    conn  = pymysql.connect(host='localhost',port=3306,user='root',password='')
    cur   = conn.cursor()
    cur.execute(query,val)
    conn.commit()
    conn.close()
    return "Done"

In [10]:
def get_gold_price_history(url):
    data = requests.get(url).text
    soup = BeautifulSoup(data)
    title=soup.title.text
    header=[]
    date=[]
    goldprice_table = soup.find('table',attrs={'id':'historymain_table_gold1'})
    for table_header in goldprice_table.findAll('th'):
         if table_header.text != '':
                header.append(table_header.text)
    for table_goldprice in goldprice_table.findAll('td'):
         if table_header.text != '':
                date.append(table_goldprice.text)
    return title,header,date


In [11]:
def get_gold_price_today(url):
    data = requests.get(url).text
    soup = BeautifulSoup(data)
    title=soup.title.text
    header=[]
    date=[]
    goldprice_table = soup.find('table',attrs={'id':'gold_price_table2'})
    for table_header in goldprice_table.findAll('th'):
         if table_header.text != '':
                header.append(table_header.text)
    for table_goldprice in goldprice_table.findAll('td'):
         if table_header.text != '':
                date.append(table_goldprice.text)
    return title,header,date

In [12]:
def gold_price_history():
    db='gold_price'
    table='history'
    db_config('localhost',3306,'root','',db,table)
    title,header,data=get_gold_price_history('http://goldpricez.com/eg/gram');
    size=int(len(data))
    for x in  range(0, size,2):
        insert_history(db,table,data[x],data[x+1])
        print (data[x],data[x+1],"inserted")

In [13]:
def gold_price_today():
    db='gold_price'
    table='today'
    db_config('localhost',3306,'root','',db,table)
    title,header,data=get_gold_price_today('http://goldpricez.com/eg/gram');
    size=int(len(data))
    for x in  range(0, size,2):
        insert_today(db,table,data[x],data[x+1])
        print (data[x],data[x+1],"inserted")
        

In [17]:
import pandas as pd
conn = pymysql.connect(host='localhost',port=3306,user='root',password='')
df = pd.read_sql_query("SELECT * FROM gold_price.today",conn)
df.head(13)

,id,date,Unit,price
0,1,2021-05-16 02:13:52,Gram 24K,927.75 EGP
1,2,2021-05-16 02:13:52,Gram 23K,889.09 EGP
2,3,2021-05-16 02:13:52,Gram 22K,850.44 EGP
3,4,2021-05-16 02:13:52,Gram 21K,811.78 EGP
4,5,2021-05-16 02:13:52,Gram 18K,695.81 EGP
5,6,2021-05-16 02:13:52,Gram 16K,618.50 EGP
6,7,2021-05-16 02:13:52,Gram 14K,541.19 EGP
7,8,2021-05-16 02:13:52,Gram 12K,463.87 EGP
8,9,2021-05-16 02:13:52,Gram 10K,386.56 EGP
9,10,2021-05-16 02:13:52,Gram 9K,347.91 EGP


In [18]:
import pandas as pd
conn = pymysql.connect(host='localhost',port=3306,user='root',password='')
df = pd.read_sql_query("SELECT * FROM gold_price.history",conn)
df.head(15)

,id,date,price
0,1,Today,927.75 EGP
1,2,Yesterday,927.89 EGP
2,3,"Fri, 14th May",927.64 EGP
3,4,"Thu, 13th May",919.81 EGP
4,5,"Wed, 12th May",917.57 EGP
5,6,"Tue, 11th May",926.93 EGP
6,7,"Mon, 10th May",926.26 EGP
7,8,"Sun, 9th May",922.14 EGP
8,9,"Sat, 8th May",922.27 EGP
9,10,"Fri, 7th May",923.00 EGP
